# Modules Import


### In this section, we import all necessary libraries for our data processing and analysis tasks. This includes libraries for handling data structures (pandas, numpy), visualization (matplotlib, seaborn), working with JSON and compressed files (json, gzip), and text processing (nltk, re)

In [8]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import gzip
import ast
import re
import dask.dataframe as dd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import gc




# Stop words download
'''
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')
'''


"\nnltk.download('vader_lexicon')\nnltk.download('stopwords')\nnltk.download('punkt')\n"

# Datasets Import


## Games Dataset

### Here, we load and preprocess the dataset containing information about various games. This involves reading from a compressed .gz file, decoding JSON objects, and cleaning the data by dropping rows that are completely null.

In [9]:
def load_and_process_games(file_path_games):

    """
    Load and process a dataset of game information stored in a JSON file.

    Args:
    file_path_games (str): The file path to the dataset in JSON format, with each game's data on a separate line.

    Returns:
    pandas.DataFrame: A processed DataFrame with selected game information and derived features.

    This function performs the following operations:
    - Reads a JSON line file into a DataFrame.
    - Removes rows and columns that contain only missing values or irrelevant information.
    - Converts the 'price' column to numeric, handling non-numeric and specific strings.
    - Parses and extracts the year from the 'release_date' and handles non-standard entries.
    - Processes the 'genres' into dummy variables and removes infrequent genre columns.
    """


    df_ga = []
    with open(file_path_games, 'r') as file:
        # Load each line of the file as a separate JSON object and append to a list
        for line in file:
            json_obj = json.loads(line)
            df_ga.append(json_obj)

    # Convert the list of dictionaries to a DataFrame
    df_ga = pd.DataFrame(df_ga)

    # Drop rows where all elements are NaN and remove specific columns
    df_ga.dropna(how='all', inplace=True)
    df_ga.drop(columns=['publisher', 'title', 'url', 'early_access', 'reviews_url', 'tags'], inplace=True)
    
    # Convert the 'price' field, replace 'Free to Play' with 0 and coerce errors
    df_ga['price'] = pd.to_numeric(df_ga['price'].replace('Free to Play', 0), errors='coerce')
    
    # Parse 'release_date', handle missing and specific string values, extract year
    df_ga['release_date'] = pd.to_datetime(df_ga['release_date'].fillna(0).replace('Soon..', 0), errors='coerce')
    df_ga['release_date'] = df_ga['release_date'].dt.year.astype('Int64')

    # Ensure 'genres' is a list, and create dummy variables for each genre
    df_ga['genres'] = df_ga['genres'].apply(lambda x: x if isinstance(x, list) else [])
    for genre in set([genre for sublist in df_ga['genres'] for genre in sublist]):
        df_ga[f'genre_{genre}'] = df_ga['genres'].apply(lambda x: 1 if genre in x else 0)

    # Remove original 'genres' column
    df_ga.drop(columns=['genres'], inplace=True)

    # Remove genre columns with fewer than 100 occurrences
    for column in df_ga.columns:
        if column.startswith('genre_') and df_ga[column].sum() < 100:
            df_ga.drop(columns=[column], inplace=True)

    return df_ga

# Usage example:
file_path_games = r"..\PI MLOps - STEAM\steam_games.json\output_steam_games.json"
df_ga = load_and_process_games(file_path_games)


## Reviews Dataset

In [10]:
# Funciones de ayuda para el procesamiento de texto y análisis de sentimientos
def preprocess_text(text):

    """
    Preprocesses a given text by converting to lowercase, removing URLs and special characters,
    tokenizing, removing stopwords, and applying stemming.

    Args:
    text (str): The input text string that needs to be processed.

    Returns:
    str: The processed text string.

    This function performs the following steps:
    - Converts the text to lowercase to standardize it.
    - Removes URLs to clean the text from any web links.
    - Removes any non-alphabetic characters (special characters and numbers) to focus on words.
    - Tokenizes the text into individual words or tokens.
    - Filters out common English stopwords to reduce noise and focuses on meaningful words.
    - Applies stemming to reduce words to their root form, enabling basic normalization.
    """

    stemmer = PorterStemmer()

    # Handle the case where the input text is None
    if text is None:
        return ""

    # Convert text to lowercase to ensure uniformity
    text = text.lower()

    # Remove URLs from the text to clean unnecessary web links
    text = re.sub(r'http\S+', '', text) 

    # Remove non-alphabetic characters to focus on words only
    text = re.sub(r'[^a-z\s]', '', text) 

    # Tokenize the text into individual words
    tokens = word_tokenize(text)

    # Remove stopwords and apply stemming to each word
    tokens = [stemmer.stem(word) for word in tokens if word not in set(stopwords.words('english'))]

    # Join the tokens back into a single string separated by space
    return ' '.join(tokens)



In [11]:
def sentiment_analysis_vader(sid, text):

    """
    Analyze the sentiment of a given text using VADER and classify it as positive, neutral, or negative.

    Args:
    sid (SentimentIntensityAnalyzer): The VADER SentimentIntensityAnalyzer instance.
    text (str): The text to analyze.

    Returns:
    int: The sentiment classification result (2 for positive, 1 for neutral, 0 for negative).

    This function performs the following operations:
    - Checks if the input text is missing (NaN) and returns 1 (neutral) if so.
    - Computes the sentiment scores using VADER's polarity_scores method.
    - Extracts the 'compound' score, which is a normalized, weighted composite score.
    - Classifies the sentiment based on the 'compound' score:
      - Positive if the compound score is 0.05 or higher.
      - Negative if the compound score is -0.05 or lower.
      - Neutral otherwise (between -0.05 and 0.05).
    """

    # Check for missing text and return neutral if no text is provided
    if pd.isna(text):
        return 1  # Neutral sentiment for missing text
    
    # Obtain polarity scores for the text from VADER
    scores = sid.polarity_scores(text)
    compound = scores['compound'] # Extract the compound score

    # Determine sentiment classification based on the compound score
    if compound >= 0.05:
        return 2  # Positive sentiment
    elif compound <= -0.05:
        return 0  # Negative sentiment
    else:
        return 1  # Neutral sentiment

In [12]:
def extract_value_from_dict(series, key):

    """
    Extract a value from a dictionary inside a pandas Series based on the provided key.

    Args:
    series (pd.Series): A pandas Series containing dictionary objects.
    key (str): The key for which the value needs to be extracted.

    Returns:
    pd.Series: A Series containing the extracted values for the specified key.
    """
        
    return series.apply(lambda x: x.get(key) if isinstance(x, dict) else None)


def load_and_process_reviews(file_path_reviews):

    """
    Load and process review data from a JSON file, extract relevant fields, apply text preprocessing,
    perform sentiment analysis, and filter the data based on specific criteria.

    Args:
    file_path_reviews (str): The file path to the reviews dataset in JSON format.

    Returns:
    pandas.DataFrame: A processed DataFrame containing cleaned reviews and their sentiment analysis results.

    This function performs the following operations:
    - Reads a JSON file line by line and parses the data.
    - Drops unnecessary columns and handles missing values.
    - Extracts specific fields from a nested dictionary within the 'reviews' column.
    - Filters out reviews based on text length.
    - Applies text preprocessing and sentiment analysis.
    - Cleans the DataFrame by removing duplicates and irrelevant rows.
    """

    df_re = []
    with open(file_path_reviews, 'r', encoding='utf-8') as file:
        # Load each line of the file and convert it to a dictionary
        for line in file:
            try:
                json_data = ast.literal_eval(line)
                df_re.append(json_data)
            except ValueError as e:
                print(f"Error in line: {line}")
                continue

    # Convert the list of dictionaries to a DataFrame
    df_re = pd.DataFrame(df_re)

    # Drop columns and rows with all elements missing
    df_re.drop(['user_url'], axis=1, inplace=True)
    df_re.dropna(how='all', inplace=True)

    # Explode 'reviews' column to expand lists into rows and reset index
    df_re = df_re.explode('reviews').reset_index(drop=True)
    
    # Extract relevant information from the 'reviews' dictionaries
    df_re['item_id'] = extract_value_from_dict(df_re['reviews'], 'item_id')
    df_re['recommend'] = extract_value_from_dict(df_re['reviews'], 'recommend')
    df_re['review_text'] = extract_value_from_dict(df_re['reviews'], 'review')

    # Drop rows where 'review_text' is missing
    df_re.dropna(subset=['review_text'], inplace=True)

    # Filter reviews to include only those with at least 5 words
    df_re = df_re[df_re['review_text'].apply(lambda x: len(x.split()) >= 5)]

    # Remove the original 'reviews' column and any duplicates
    df_re.drop(columns=['reviews'], inplace=True)
    df_re.drop_duplicates(inplace=True)

    # Perform sentiment analysis on cleaned reviews
    sid = SentimentIntensityAnalyzer()
    df_re['cleaned_review'] = df_re['review_text'].apply(preprocess_text)
    df_re['sentiment_analysis'] = df_re['cleaned_review'].apply(lambda text: sentiment_analysis_vader(sid, text))

    # Drop rows where 'sentiment_analysis' results are missing
    df_re.dropna(subset=['sentiment_analysis'], inplace=True)

    return df_re

# File path for the JSON dataset
file_path_reviews = r"..\PI MLOps - STEAM\user_reviews.json\australian_user_reviews.json"

# Processing and storing the reviews dataset
df_re = load_and_process_reviews(file_path_reviews)


#### Similarly, for the reviews dataset, we process each line from a JSON file, handling potential errors and converting lines into a list of dictionaries, which we then transform into a DataFrame. This step is crucial for structuring the reviews data for further analysis.

#### In order to analyze the sentiment of user reviews, we first preprocess the text to remove URLs, special characters, and numbers, and to tokenize the text. This preprocessing step is vital for reducing noise in the text data and improving the performance of our sentiment analysis.

#### We utilize the VADER tool from the nltk library to perform sentiment analysis on the preprocessed review texts. This tool is particularly suited for texts from social media and similar contexts due to its sensitivity to both the polarity and intensity of emotions.



### Items Dateset

#### In this section, we focus on the dataset that contains information about users' items. This includes the games or software owned by users on the Steam platform. We start by loading the data from a JSON file and proceed to clean it by removing entries that don't provide meaningful information.

In [13]:
def load_and_process_items(file_path_items):

    """
    Load and process item data from a JSON file, extract relevant fields, and filter the data based on specific criteria.

    Args:
    file_path_items (str): The file path to the items dataset in JSON format.

    Returns:
    pandas.DataFrame: A processed DataFrame containing item data with filters applied to playtime.

    This function performs the following operations:
    - Reads a JSON file line by line and parses the data into a list of dictionaries.
    - Converts the list into a DataFrame and drops unnecessary columns.
    - Filters out entries with a zero item count and expands lists of items into separate rows.
    - Extracts relevant information from the nested dictionaries within the 'items' column.
    - Filters items based on the 'playtime_forever' to include only those with less than 2 hours.
    - Removes the original 'items' column and any duplicate entries.

    The function is particularly useful for cleaning and preparing game user data for analysis,
    focusing on user engagement measured through playtime.
    """

    df_it = []
    with open(file_path_items, 'r', encoding='utf-8') as file:
        # Load each line of the file, parse as a dictionary, and append to a list
        for line in file:
            try:
                json_data = ast.literal_eval(line)
                df_it.append(json_data)
            except ValueError as e:
                print(f"Error in line: {line}")
                continue

    # Convert the list of dictionaries to a DataFrame
    df_it = pd.DataFrame(df_it)

    # Drop columns and filter rows where 'items_count' is zero
    df_it.drop(['user_url'], axis=1, inplace=True)
    df_it = df_it[df_it['items_count'] != 0].reset_index(drop=True)

    # Explode 'items' column to separate rows and reset index
    df_it = df_it.explode('items').reset_index(drop=True)
    
    # Extract 'item_id' and 'playtime_forever' from the dictionaries in the 'items' column
    df_it['item_id'] = df_it['items'].apply(lambda x: x.get('item_id') if isinstance(x, dict) else None)
    df_it['playtime_forever'] = df_it['items'].apply(lambda x: x.get('playtime_forever') if isinstance(x, dict) else None)

    # Filter items to include only those with less than 2 hours of playtime
    df_it = df_it[df_it['playtime_forever'] < 7200]    
    
    # Remove the original 'items' column and any duplicates
    df_it.drop(columns=['items'], inplace=True)
    df_it.drop_duplicates(inplace=True)
   
    return df_it


# File path for the JSON dataset
file_path_items = r"..\PI MLOps - STEAM\users_items.json\australian_users_items.json"

# Processing and storing the items dataset
df_it = load_and_process_items(file_path_items)


### Precalculated Datesets for API feeding

In [14]:
def precalculate_developer_stats(df_ga):

    """
    Calculate statistics for each developer in the dataset, including the total number of games and the percentage of games that are free.

    Args:
    df_ga (pd.DataFrame): A DataFrame containing game data with at least 'developer', 'id', and 'price' columns.

    This function performs the following operations:
    - Groups the data by the 'developer' column.
    - Aggregates two statistics for each developer:
      * The total number of games developed.
      * The count of games that are free (price == 0).
    - Calculates the percentage of games that are free for each developer.
    - Saves the resulting statistics DataFrame to a Parquet file for efficient storage and access.

    The use of Parquet format is ideal for performance in both storage and computational efficiency, especially when dealing with large datasets.
    """

    # Group the DataFrame by 'developer' and aggregate the necessary statistics
    developer_stats = df_ga.groupby('developer').agg(
        total_games=('id', 'count'), # Count the number of games per developer
        free_games=('price', lambda x: (x == 0).sum()) # Count how many games are free
    )

    # Calculate the percentage of free games for each developer
    developer_stats['free_game_percentage'] = (developer_stats['free_games'] / developer_stats['total_games']) * 100

    # Save the aggregated data to a Parquet file
    developer_stats.to_parquet ('../datasets/developer_stats.parquet')



In [15]:
def precalculate_review_analysis(df_ga, df_re):

    """
    Calculate and save a summary of sentiment analysis results for each developer based on reviews of their games.

    Args:
    df_ga (pd.DataFrame): A DataFrame containing game data with at least 'id' and 'developer' columns.
    df_re (pd.DataFrame): A DataFrame containing review data with at least 'item_id' and 'sentiment_analysis' columns.

    This function performs the following operations:
    - Merges review data with game data to associate reviews with the developers of the games.
    - Aggregates sentiment analysis results by developer.
    - Saves the aggregated data to a Parquet file for efficient storage and access.

    The analysis groups sentiments into categories (e.g., positive, neutral, negative) and counts occurrences for each category per developer. This allows quick retrieval and analysis of sentiment data for each developer, facilitating user feedback analysis at a granular level.
    """

    # Merge review data with game data to map each review to its respective developer
    df_re = df_re.merge(df_ga[['id', 'developer']], left_on='item_id', right_on='id')

    # Aggregate sentiment analysis results by developer and fill missing values with 0 (for developers with no reviews in a particular category)
    review_analysis = df_re.groupby('developer')['sentiment_analysis'].value_counts().unstack().fillna(0)

    # Save the results to a Parquet file
    review_analysis.to_parquet('../datasets/review_analysis.parquet')



In [16]:
# Precalculo de datos para Endpoint 3
def precalculate_genre_playtime_stats(df_it, df_ga):

    """
    Calculate and save average playtime statistics for each genre and overall, using game item and game data.

    Args:
    df_it (pd.DataFrame): A DataFrame containing item data with at least 'item_id' and 'playtime_forever' columns.
    df_ga (pd.DataFrame): A DataFrame containing game data with 'id' and genre-specific columns.

    This function performs the following operations:
    - Identifies genre columns in the game data.
    - Merges item data with game data to link playtime with game genres.
    - Computes the average playtime for each genre and the overall average playtime across all genres.
    - Saves the results to a Parquet file for efficient storage and access.

    These statistics provide insights into user engagement across different game genres, facilitating targeted marketing and game development strategies.
    """

    # Identify genre columns in the game DataFrame
    genre_columns = [col for col in df_ga.columns if col.startswith('genre_')]

    # Merge the item and game DataFrames to correlate items with their game genre
    merged_df = df_it.merge(df_ga, left_on='item_id', right_on='id')

    # Initialize a dictionary to store playtime statistics for each genre
    genre_playtime_stats = {}

    # Calculate the average playtime for each genre
    for genre_col in genre_columns:
        # Only consider rows where the genre is applicable
        average_playtime = merged_df[merged_df[genre_col] == 1]['playtime_forever'].mean()
        genre = genre_col.split('_')[1] # Extract the genre name from the column
        genre_playtime_stats[genre] = average_playtime

    # Calculate the overall average playtime across all genres
    total_average_playtime = merged_df['playtime_forever'].mean()

    # Convert the dictionary of genre playtimes to a DataFrame
    genre_playtime_df = pd.DataFrame.from_dict(genre_playtime_stats, orient='index', columns=['average_playtime'])
    genre_playtime_df['total_average_playtime'] = total_average_playtime

    # Save the DataFrame as a Parquet file for efficient storage and querying
    genre_playtime_df.to_parquet('../datasets/genre_playtime_stats.parquet')



In [17]:
# Precalculo de datos para Endpoint 4
def precalculate_user_data(df_ga, df_re, df_it):

    """
    Aggregate user data across multiple datasets to calculate spending, review counts, and recommendation percentages.

    Args:
    df_ga (pd.DataFrame): DataFrame containing game data, specifically game ids and prices.
    df_re (pd.DataFrame): DataFrame containing review data, specifically item ids and recommendations.
    df_it (pd.DataFrame): DataFrame containing user-item interactions, including user ids and item ids.

    This function performs the following operations:
    - Converts the Pandas DataFrames to Dask DataFrames to utilize parallel processing for handling larger data volumes.
    - Merges these DataFrames to align user-item interactions with game prices and reviews.
    - Aggregates data to calculate the total spent, total reviews, and number of recommendations per user.
    - Computes the recommendation percentage for each user.
    - Saves the aggregated user data to a Parquet file for efficient storage and querying.

    The use of Dask allows for efficient computation on larger datasets that might not fit into memory if using Pandas alone.
    """

    # Convert Pandas DataFrames to Dask DataFrames for parallel processing
    ddf_it = dd.from_pandas(df_it, npartitions=15)
    
    # Merge the dataframes using Dask to handle large datasets
    df_merged = ddf_it.merge(df_ga[['id', 'price']], left_on='item_id', right_on='id', how='left')
    df_merged = df_merged.merge(df_re[['item_id', 'recommend']], on='item_id', how='left')
    
    # Aggregate data using Dask, which is optimized for big data processing
    df_user_stats = df_merged.groupby('user_id').agg({
        'price': 'sum',
        'recommend': 'count',
        'item_id': 'count'
    }).compute()  # Trigger computations and bring results back to Pandas for further operations

    # Rename columns for clarity and calculate the recommendation percentage
    df_user_stats = df_user_stats.rename(columns={
        'price': 'total_spent',
        'recommend': 'recommendations',
        'item_id': 'total_reviews'
    })
    df_user_stats['recommendation_percentage'] = (df_user_stats['recommendations'] / df_user_stats['total_reviews']) * 100
    
    # Save the resulting DataFrame to a Parquet
    df_user_stats.to_parquet('../datasets/user_data.parquet')


In [18]:
# Precalculo de datos para Endpoint 5
def precalculate_best_developers_by_year(df_ga, df_re):

    """
    Calculate and save the most popular game developers by year based on the number of positive reviews.

    Args:
    df_ga (pd.DataFrame): DataFrame containing game data, specifically game ids, developers, and release dates.
    df_re (pd.DataFrame): DataFrame containing review data, specifically item ids and sentiment analysis results.

    This function performs the following operations:
    - Merges game data with review data to correlate games with their reviews.
    - Filters for positive reviews only (where sentiment_analysis == 2).
    - Aggregates the count of positive reviews by developer and release year.
    - Identifies the developer with the highest number of positive reviews for each year.
    - Saves this information in a Parquet file for efficient storage and quick access.

    This data can be particularly useful for marketing analysis, trend tracking, and recognizing industry leaders.
    """

    # Merge game and review datasets to align reviews with corresponding game details
    combined_df = df_ga.merge(df_re, left_on='id', right_on='item_id')

    # Filter for positive reviews only
    positive_reviews = combined_df[combined_df['sentiment_analysis'] == 2]

    # Count positive reviews per developer and release year
    top_devs_by_year = positive_reviews.groupby(['developer', 'release_date']).size()

    # Identify the developer with the most positive reviews each year
    top_devs_by_year = top_devs_by_year.reset_index(name='positive_reviews')
    top_devs_by_year = top_devs_by_year.sort_values(['release_date', 'positive_reviews'], ascending=False)
    top_devs_by_year = top_devs_by_year.groupby('release_date').first().reset_index()

    # Ensure the correct order of columns
    top_devs_by_year = top_devs_by_year[['release_date', 'developer']]

    # Save the result to a Parquet file
    top_devs_by_year.to_parquet(f'../datasets/best_developers_by_year.parquet')

In [19]:
def precalculate_recommendation_data(df_ga, df_re, df_it):
    """
    Create a recommendation dataset by merging game, review, and user-item interaction data,
    and aggregating necessary information for recommendation purposes.

    Args:
    df_ga (pd.DataFrame): DataFrame containing game data with game IDs and other game-related attributes.
    df_re (pd.DataFrame): DataFrame containing review data with game IDs and sentiment analysis results.
    df_it (pd.DataFrame): DataFrame containing user-item interaction data, including playtime.

    This function performs the following operations:
    - Converts Pandas DataFrames to Dask DataFrames for efficient parallel processing.
    - Merges these DataFrames to align games with their reviews and user-item interactions.
    - Filters and retains columns of interest for the recommendation system, such as game genres and user activities.
    - Groups data by game titles to aggregate genre data and compute average playtime.
    - Saves the aggregated data to a Parquet file with GZIP compression for efficient storage and retrieval.

    The function is designed to provide a structured dataset that supports building or improving game recommendation systems.
    """
    # Convert Pandas DataFrames to Dask DataFrames for parallel processing
    ddf_ga = dd.from_pandas(df_ga, npartitions=10)
    ddf_re = dd.from_pandas(df_re, npartitions=10)
    ddf_it = dd.from_pandas(df_it, npartitions=102)

    # Merge DataFrames to combine game data with reviews and user interactions
    ddf_combined = ddf_ga.merge(ddf_re, left_on='id', right_on='item_id', how='left')
    ddf_combined = ddf_combined.merge(ddf_it, left_on='id', right_on='item_id', how='left')

    # Define columns of interest for the recommendation system
    columns_of_interest = ['id', 'app_name'] + \
                          [col for col in df_ga.columns if 'genre_' in col] + \
                          ['user_id', 'sentiment_analysis', 'playtime_forever']
    columns_to_keep = [col for col in columns_of_interest if col in ddf_combined.columns]
    ddf_combined = ddf_combined[columns_to_keep]

    # Aggregate genre data and calculate average playtime by game title
    ddf_grouped = ddf_combined.groupby('app_name').sum()
    ddf_grouped['avg_playtime_forever'] = ddf_combined.groupby('app_name')['playtime_forever'].mean()

    # Repartition the DataFrame to a single partition for output
    ddf_grouped = ddf_grouped.repartition(npartitions=1)

    # Save the processed data to a Parquet file, using GZIP compression
    ddf_grouped.to_parquet('../datasets/recommendation_dataset.parquet', write_index=False, compression='gzip')


In [20]:
'''This section of the code is responsible for invoking a series of precalculation functions that have been defined above.
Each function processes and aggregates data across different aspects of game analytics to prepare datasets for advanced analysis and operations. 
The functions being called are:
- precalculate_developer_stats: Processes and saves developer-specific game statistics.
- precalculate_review_analysis: Analyzes and aggregates review data by developers.
- precalculate_genre_playtime_stats: Calculates and stores playtime statistics by game genre.
- precalculate_user_data: Aggregates comprehensive user data from game interactions and reviews.
- precalculate_best_developers_by_year: Determines and stores the most successful game developers of each year based on review data.
- precalculate_recommendation_data: Creates a dataset optimized for game recommendation algorithms.

Each function is designed to enhance data accessibility and analytical readiness, feeding into subsequent analytical tasks or systems.
'''

precalculate_developer_stats(df_ga)
precalculate_review_analysis(df_ga, df_re)
precalculate_genre_playtime_stats(df_it, df_ga)
precalculate_user_data(df_ga, df_re, df_it)
precalculate_best_developers_by_year(df_ga, df_re)
#precalculate_recommendation_data(df_ga, df_re, df_it)